# Initiation    


### Packages

In [1]:
import logging
import sys
import os
import time
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
from datetime import datetime, timedelta
import os
import sys

### Define Logger

In [2]:

# Configure logger
def setup_logger(debug_mode=False):
    level = logging.DEBUG if debug_mode else logging.INFO
    
    logging.basicConfig(
        level=level,
        format='%(asctime)s - %(name)s - %(levelname)s - %(message)s',
        handlers=[
            logging.StreamHandler(sys.stdout)  # Print to console
        ]
    )
    return logging.getLogger(__name__)


# Initialize logger
DEBUG = True  # Set to False in production
logger = setup_logger(debug_mode=DEBUG)
time_sleep=1


### define env variables

In [3]:

email = os.getenv('YOUR_SECRET_EMAIL')
password = os.getenv('YOUR_SECRET_PASSWORD')
logon_url = os.getenv("YOUR_SECRET_LOGON_URL")
planning_url = os.getenv('YOUR_SECRET_PLANNING_URL')
login_url = os.getenv('YOUR_SECRET_LOGIN_URL')
my_name = os.getenv('YOUR_SECRET_MY_NAME')


In [4]:


# Vérification des secrets
logger.info("=== STATUS DES AUTRES SECRETS (MASQUÉS) ===")
logger.info(f"EMAIL: {'✅ DÉFINI' if email else '❌ MISSING'}")
logger.info(f"PASSWORD: {'✅ DÉFINI' if password else '❌ MISSING'}")
logger.info(f"MY_NAME: {'✅ DÉFINI' if my_name else '❌ MISSING'}")
logger.info(f"LOGON_URL: {'✅ DÉFINI' if logon_url else '❌ MISSING'}")
logger.info(f"PLANNING_URL: {'✅ DÉFINI' if planning_url else '❌ MISSING'}")
logger.info(f"LOGIN_URL: {'✅ DÉFINI' if login_url else '❌ MISSING'}")

# Vérification complète
secrets_dict = {
    "EMAIL": email,
    "PASSWORD": password,
    "MY_NAME": my_name,
    "LOGON_URL": logon_url,
    "PLANNING_URL": planning_url,
    "LOGIN_URL": login_url
}

missing_secrets = [name for name, value in secrets_dict.items() if not value]

if missing_secrets:
    logger.error("❌ ERREUR: UN OU PLUSIEURS SECRETS SONT MANQUANTS")
    logger.error("=== SECRETS MANQUANTS ===")
    for secret_name in missing_secrets:
        logger.error(f"{secret_name}: ❌ MISSING")
    logger.critical("ARRÊT DU PROGRAMME - VÉRIFIEZ VOS VARIABLES D'ENVIRONNEMENT")
    sys.exit(1)
else:
    logger.info("✅ TOUS LES SECRETS SONT DÉFINIS")
    logger.debug("Poursuite de l'exécution...")

2025-11-01 22:53:24,969 - __main__ - INFO - === STATUS DES AUTRES SECRETS (MASQUÉS) ===
2025-11-01 22:53:24,981 - __main__ - INFO - EMAIL: ✅ DÉFINI
2025-11-01 22:53:24,983 - __main__ - INFO - PASSWORD: ✅ DÉFINI
2025-11-01 22:53:24,984 - __main__ - INFO - MY_NAME: ✅ DÉFINI
2025-11-01 22:53:24,985 - __main__ - INFO - LOGON_URL: ✅ DÉFINI
2025-11-01 22:53:24,986 - __main__ - INFO - PLANNING_URL: ✅ DÉFINI
2025-11-01 22:53:24,989 - __main__ - INFO - LOGIN_URL: ✅ DÉFINI
2025-11-01 22:53:24,994 - __main__ - INFO - ✅ TOUS LES SECRETS SONT DÉFINIS
2025-11-01 22:53:24,998 - __main__ - DEBUG - Poursuite de l'exécution...


# code

In [5]:


class TennisBookingBot:
    """Bot de réservation de tennis/sport complet avec screenshots debug"""
    
    def __init__(self, target_date, target_time, course_level, player_name, time_sleep=1, debug_mode=False):
        # Paramètres de réservation
        self.target_date = target_date
        self.target_time = target_time
        self.course_level = course_level
        self.player_name = player_name
        self.time_sleep = time_sleep
        self.debug_mode = debug_mode
        self.screenshot_counter = 0
        
        # URLs et credentials depuis env
        self.email = os.getenv('YOUR_SECRET_EMAIL')
        self.password = os.getenv('YOUR_SECRET_PASSWORD')
        self.logon_url = os.getenv("YOUR_SECRET_LOGON_URL")
        self.planning_url = os.getenv('YOUR_SECRET_PLANNING_URL')
        self.login_url = os.getenv('YOUR_SECRET_LOGIN_URL')
        
        # Driver
        self.driver = None
        self.logged_in = False
        
        # Vérifier les secrets
        self._check_secrets()
        
        logger.info(f"🤖 Bot créé pour:")
        logger.info(f"📅 Date: {self.target_date}")
        logger.info(f"🕐 Time: {self.target_time}")
        logger.info(f"🎾 Level: {self.course_level}")
        logger.info(f"👤 Player: {self.player_name}")
        logger.info(f"🐛 Debug mode: {'✅ ON' if self.debug_mode else '❌ OFF'}")
    
    def _check_secrets(self):
        """Vérifier que tous les secrets sont présents"""
        secrets = {
            "EMAIL": self.email,
            "PASSWORD": self.password,
            "LOGON_URL": self.logon_url,
            "PLANNING_URL": self.planning_url,
            "LOGIN_URL": self.login_url
        }
        
        logger.info("=== STATUS DES SECRETS ===")
        missing = []
        for name, value in secrets.items():
            status = "✅ DÉFINI" if value else "❌ MISSING"
            logger.info(f"{name}: {status}")
            if not value:
                missing.append(name)
        
        if missing:
            logger.error(f"❌ ERREUR: {len(missing)} secrets manquants")
            sys.exit(1)
        logger.info("✅ Tous les secrets OK")
    
    def _debug_screenshot(self, step_name):
        """Prendre une screenshot en mode debug"""
        if self.debug_mode and self.driver:
            self.screenshot_counter += 1
            filename = f"debug_{self.screenshot_counter:02d}_{step_name}_{self.target_date.replace('-', '_')}.png"
            self.driver.save_screenshot(filename)
            logger.debug(f"📸 Screenshot: {filename}")
            return filename
        return None
    
    def _setup_driver(self):
        """Initialise le driver Chrome"""
        logger.info("🚀 Initialisation du driver...")
        options = Options()
        options.add_argument('--headless')
        options.add_argument('--no-sandbox')
        
        service = Service(ChromeDriverManager().install())
        self.driver = webdriver.Chrome(service=service, options=options)
        self.driver.set_window_size(1400, 900)
        logger.info("✅ Driver configuré")
        self._debug_screenshot("driver_setup")
    
    def login(self):
        """Se connecter au site"""
        if not self.driver:
            self._setup_driver()
        
        logger.info("🔐 Connexion en cours...")
        self.driver.get(self.logon_url)
        logger.info(f"📄 Titre: {self.driver.title}")
        self._debug_screenshot("login_page_loaded")
        
        wait = WebDriverWait(self.driver, 10)
        try:
            # Email
            email_field = wait.until(EC.visibility_of_element_located((By.XPATH, "//input[@type='email']")))
            email_field.clear()
            email_field.send_keys(self.email)
            logger.info("📧 Email saisi")
            self._debug_screenshot("email_entered")
            
            # Password
            password_field = wait.until(EC.visibility_of_element_located((By.XPATH, "//input[@type='password']")))
            password_field.clear()
            password_field.send_keys(self.password)
            logger.info("🔑 Password saisi")
            self._debug_screenshot("password_entered")
            
            # Login button
            login_button = wait.until(EC.element_to_be_clickable((By.XPATH, "//input[@value='Login']")))
            login_button.click()
            logger.info("✅ Bouton Login cliqué")
            self._debug_screenshot("login_button_clicked")
            
            # Attendre redirection
            wait.until(EC.url_changes(self.logon_url))
            self.logged_in = True
            logger.info(f"🌍 Connecté: {self.driver.current_url}")
            self._debug_screenshot("login_success")
            return True
            
        except TimeoutException as e:
            logger.error(f"❌ Erreur login: {e}")
            self._debug_screenshot("login_error")
            return False
    
    def _check_date_validity(self):
        """Vérifier si la date est valide pour réservation"""
        try:
            target_datetime = datetime.strptime(self.target_date, "%d-%b-%y")
            today = datetime.now()
            difference = target_datetime - today
            days_difference = difference.days
            
            is_future = target_datetime > today
            is_within_week = days_difference <= 6
            is_next_week = days_difference == 6
            
            logger.info(f"📅 Date cible: {target_datetime.strftime('%Y-%m-%d')}")
            logger.info(f"📅 Aujourd'hui: {today.strftime('%Y-%m-%d')}")
            logger.info(f"📊 Différence: {days_difference} jours")
            
            can_book = is_future and is_within_week
            return can_book, is_next_week
            
        except ValueError:
            logger.error(f"❌ Format de date invalide: {self.target_date}")
            return False, False
    
    def _check_basket_empty(self):
        """Vérifier si le panier est vide"""
        self._debug_screenshot("before_basket_check")
        if self.driver.current_url == self.login_url:
            is_empty = self.driver.execute_script("return document.querySelector('span.basket-badge').textContent;") == "0"
            logger.info(f"🛒 Panier {'vide' if is_empty else 'non vide'}")
            self._debug_screenshot("basket_checked")
            return is_empty
        logger.warning("❌ Pas sur la page de login")
        self._debug_screenshot("not_on_login_page")
        return False
    
    def _navigate_to_planning(self, next_week=False):
        """Naviguer vers la page de planning"""
        logger.info("🗓️ Navigation vers planning...")
        self.driver.get(self.planning_url)
        time.sleep(self.time_sleep)
        self._debug_screenshot("planning_page_loaded")
        
        if next_week:
            logger.info("⏭️ Navigation vers semaine suivante...")
            next_week_btn = self.driver.find_element(By.XPATH, "//input[@value='Next Week']")
            next_week_btn.click()
            time.sleep(self.time_sleep)
            self._debug_screenshot("next_week_clicked")
        
        logger.info("✅ Sur la page planning")
        self._debug_screenshot("final_planning_view")
    
    def _find_available_slot(self):
        """Chercher un créneau disponible"""
        logger.info("🔍 Recherche de créneaux...")
        self._debug_screenshot("before_slot_search")
        
        course_buttons = self.driver.find_elements(By.XPATH, "//button[@data-class-time]")
        logger.info(f"📋 {len(course_buttons)} boutons trouvés")
        
        for i, button in enumerate(course_buttons):
            # Extraire les données
            class_name = button.get_attribute('data-class-name') or ''
            class_date = button.get_attribute('data-class-date') or ''
            class_time = button.get_attribute('data-class-time') or ''
            class_spaces = button.get_attribute('data-class-spaces') or '0'
            
            # Vérifier correspondance
            date_match = self.target_date in class_date
            level_match = self.course_level in class_name
            time_match = self.target_time in class_time.split("-")[0] or self.target_time.strip() in class_time.split("-")[0]
            
            logger.debug(f"Bouton {i}: {class_name} | {class_date} | {class_time} | Espaces: {class_spaces}")
            logger.debug(f"  Date match: {date_match} | Level match: {level_match} | Time match: {time_match}")
            
            if date_match and time_match and level_match:
                spaces_available = int(class_spaces)
                logger.info(f"✅ CRÉNEAU TROUVÉ!")
                logger.info(f"  📛 Nom: {class_name}")
                logger.info(f"  📅 Date: {class_date}")
                logger.info(f"  🕐 Heure: {class_time}")
                logger.info(f"  👥 Places: {spaces_available}")
                
                self._debug_screenshot("slot_found")
                
                if spaces_available > 0:
                    return {
                        'button': button,
                        'available': True,
                        'spaces': spaces_available
                    }
                else:
                    logger.warning("❌ Créneau complet")
                    self._debug_screenshot("slot_full")
                    return {
                        'button': button,
                        'available': False,
                        'spaces': 0
                    }
        
        logger.warning("❌ Aucun créneau trouvé")
        self._debug_screenshot("no_slot_found")
        return None
    
    def _click_book_slot(self, slot):
        """Cliquer pour réserver le créneau"""
        logger.info("📝 Réservation du créneau...")
        self._debug_screenshot("before_book_click")
        
        parent = slot['button'].find_element(By.XPATH, "./..")
        book_buttons = parent.find_elements(By.XPATH, ".//*[contains(text(), 'Book Now') or contains(text(), 'Book')]")
        
        if book_buttons:
            logger.info("✅ Bouton 'Book' trouvé")
            book_buttons[0].click()
            time.sleep(self.time_sleep)
            self._debug_screenshot("book_button_clicked")
            return True
        
        logger.error("❌ Bouton 'Book' non trouvé")
        self._debug_screenshot("book_button_not_found")
        return False
    
    def _select_player(self):
        """Sélectionner le joueur"""
        logger.info(f"👤 Sélection du joueur: {self.player_name}")
        self._debug_screenshot("before_player_selection")
        
        player_elements = self.driver.find_elements(By.XPATH, f"//*[contains(text(), '{self.player_name}')]")
        
        for player_element in player_elements:
            parent = player_element.find_element(By.XPATH, "./..")
            select_buttons = parent.find_elements(By.XPATH, ".//*[contains(text(), 'Book') or contains(text(), 'Select') or contains(text(), 'Choose')]")
            
            if select_buttons:
                logger.info(f"✅ Bouton de sélection trouvé pour {self.player_name}")
                select_buttons[0].click()
                time.sleep(self.time_sleep)
                self._debug_screenshot("player_selected")
                return True
        
        logger.error(f"❌ Impossible de sélectionner {self.player_name}")
        self._debug_screenshot("player_selection_failed")
        return False
    
    def _confirm_booking(self):
        """Confirmer la réservation"""
        logger.info("✅ Confirmation de la réservation...")
        self._debug_screenshot("before_checkout")
        
        checkout_buttons = self.driver.find_elements(By.XPATH, "//*[contains(text(), 'Checkout')]")
        if checkout_buttons:
            checkout_buttons[0].click()
            time.sleep(self.time_sleep)
            self._debug_screenshot("checkout_clicked")
            logger.info("🎉 RÉSERVATION CONFIRMÉE!")
            self._debug_screenshot("booking_confirmed")
            return True
        
        logger.error("❌ Bouton Checkout non trouvé")
        self._debug_screenshot("checkout_not_found")
        return False
    
    def book(self):
        """Méthode principale de réservation"""
        if not self.logged_in:
            logger.error("❌ Pas connecté. Appelez login() d'abord")
            return False
        
        logger.info("🎯 Début de la réservation...")
        self._debug_screenshot("booking_start")
        
        # 1. Vérifier la date
        can_book, next_week = self._check_date_validity()
        if not can_book:
            logger.error("❌ Date non valide pour réservation")
            self._debug_screenshot("invalid_date")
            return False
        
        # 2. Vérifier panier vide
        if not self._check_basket_empty():
            logger.error("❌ Panier non vide")
            return False
        
        # 3. Aller au planning
        self._navigate_to_planning(next_week)
        
        # 4. Chercher créneau
        slot = self._find_available_slot()
        if not slot:
            logger.error("❌ Aucun créneau disponible")
            return False
        
        if not slot['available']:
            logger.error("❌ Créneau complet")
            return False
        
        # 5. Réserver le créneau
        if not self._click_book_slot(slot):
            return False
        
        # 6. Sélectionner joueur
        if not self._select_player():
            return False
        
        # 7. Confirmer
        success = self._confirm_booking()
        if success:
            self._debug_screenshot("final_success")
        else:
            self._debug_screenshot("final_failure")
        
        return success
    
    def quit(self):
        """Fermer le driver"""
        if self.driver:
            self._debug_screenshot("before_quit")
            self.driver.quit()
            logger.info("🔒 Driver fermé")



In [6]:
debug_mode=True


In [24]:
# Valeurs par défaut
DEFAULT_DATE = "02-Nov-25"
DEFAULT_TIME = "9:00"
DEFAULT_LEVEL = "Novice"
DEFAULT_NAME = os.getenv('YOUR_SECRET_MY_NAME', 'Player')

In [ ]:
# Arguments ou défauts
target_date = sys.argv[1] if len(sys.argv) > 1 else DEFAULT_DATE
target_time = sys.argv[2] if len(sys.argv) > 2 else DEFAULT_TIME
course_level = sys.argv[3] if len(sys.argv) > 3 else DEFAULT_LEVEL
player_name = sys.argv[4] if len(sys.argv) > 4 else DEFAULT_NAME


In [ ]:
debug_mode=True
# Valeurs par défaut
target_date = "02-Nov-25"
target_time = "9:00"
course_level = "Novice"
DEFAULT_NAME = os.getenv('YOUR_SECRET_MY_NAME', 'Player')
player_name ='Serge Lebon'

In [26]:
target_date

'02-Nov-25'

In [27]:
    # Créer bot avec mode debug
bot = TennisBookingBot(
        target_date=target_date,
        target_time=target_time, 
        course_level=course_level,
        player_name=player_name,
        debug_mode=debug_mode
    )

2025-11-01 23:01:11,983 - __main__ - INFO - === STATUS DES SECRETS ===
2025-11-01 23:01:11,989 - __main__ - INFO - EMAIL: ✅ DÉFINI
2025-11-01 23:01:11,992 - __main__ - INFO - PASSWORD: ✅ DÉFINI
2025-11-01 23:01:11,994 - __main__ - INFO - LOGON_URL: ✅ DÉFINI
2025-11-01 23:01:11,996 - __main__ - INFO - PLANNING_URL: ✅ DÉFINI
2025-11-01 23:01:11,997 - __main__ - INFO - LOGIN_URL: ✅ DÉFINI
2025-11-01 23:01:11,998 - __main__ - INFO - ✅ Tous les secrets OK
2025-11-01 23:01:11,999 - __main__ - INFO - 🤖 Bot créé pour:
2025-11-01 23:01:11,999 - __main__ - INFO - 📅 Date: 02-Nov-25
2025-11-01 23:01:12,002 - __main__ - INFO - 🕐 Time: 9:00
2025-11-01 23:01:12,004 - __main__ - INFO - 🎾 Level: Novice
2025-11-01 23:01:12,005 - __main__ - INFO - 👤 Player: Serge Lebon
2025-11-01 23:01:12,006 - __main__ - INFO - 🐛 Debug mode: ✅ ON


In [28]:


try:
    if bot.login():
        success = bot.book()
        exit_code = 0 if success else 1
    else:
        exit_code = 1
finally:
    bot.quit()

sys.exit(exit_code)

2025-11-01 23:01:16,325 - __main__ - INFO - 🚀 Initialisation du driver...
2025-11-01 23:01:16,331 - WDM - INFO - ====== WebDriver manager ======
2025-11-01 23:01:17,111 - WDM - INFO - Get LATEST chromedriver version for google-chrome
2025-11-01 23:01:17,117 - urllib3.connectionpool - DEBUG - Starting new HTTPS connection (1): googlechromelabs.github.io:443
2025-11-01 23:01:17,271 - urllib3.connectionpool - DEBUG - https://googlechromelabs.github.io:443 "GET /chrome-for-testing/latest-patch-versions-per-build.json HTTP/1.1" 200 13887
2025-11-01 23:01:17,330 - WDM - INFO - Get LATEST chromedriver version for google-chrome
2025-11-01 23:01:17,334 - urllib3.connectionpool - DEBUG - Starting new HTTPS connection (1): googlechromelabs.github.io:443
2025-11-01 23:01:17,461 - urllib3.connectionpool - DEBUG - https://googlechromelabs.github.io:443 "GET /chrome-for-testing/latest-patch-versions-per-build.json HTTP/1.1" 200 13887
2025-11-01 23:01:17,471 - WDM - INFO - Driver [/root/.wdm/drivers/c

SystemExit: 0

/usr/local/lib/python3.11/site-packages/IPython/core/interactiveshell.py:3707: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [18]:
bot.quit()

2025-11-01 22:58:17,711 - selenium.webdriver.remote.remote_connection - DEBUG - GET http://localhost:53361/session/db6d12d8b8866b553be6327b0ef2272d/screenshot {}
2025-11-01 22:58:17,720 - urllib3.connectionpool - DEBUG - Starting new HTTP connection (1): localhost:53361
2025-11-01 22:58:17,733 - urllib3.util.retry - DEBUG - Incremented Retry for (url='/session/db6d12d8b8866b553be6327b0ef2272d/screenshot'): Retry(total=2, connect=None, read=None, redirect=None, status=None)
2025-11-01 22:58:17,736 - urllib3.connectionpool - WARNING - Retrying (Retry(total=2, connect=None, read=None, redirect=None, status=None)) after connection broken by 'NewConnectionError('<urllib3.connection.HTTPConnection object at 0x7ffff09efc50>: Failed to establish a new connection: [Errno 111] Connection refused')': /session/db6d12d8b8866b553be6327b0ef2272d/screenshot
2025-11-01 22:58:17,740 - urllib3.connectionpool - DEBUG - Starting new HTTP connection (2): localhost:53361
2025-11-01 22:58:17,744 - urllib3.uti

MaxRetryError: HTTPConnectionPool(host='localhost', port=53361): Max retries exceeded with url: /session/db6d12d8b8866b553be6327b0ef2272d/screenshot (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x7ffff09d8d90>: Failed to establish a new connection: [Errno 111] Connection refused'))